In [114]:
import numpy as np 
import pandas as pd 
import gc
import matplotlib.pyplot as plt
import time
import seaborn as sns
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.model_selection import GroupKFold
from pandas.core.common import SettingWithCopyWarning
import warnings
warnings.simplefilter('error', SettingWithCopyWarning)

gc.enable()

In [2]:
path='/home/baitong/pywork/RevenuePrediction/'

In [15]:
# def readAllData(csvpath=path,n=18,name = 'train0'):
#     fulldata = pd.DataFrame()
#     for i in range(n):       
#         print('loading data : ' + name + '%d'%i)
#         df = pd.read_csv(csvpath+name +str(i) +'.csv',dtype={'fullVisitorId': 'str'})
#         fulldata = pd.concat([fulldata,df],axis=0)
#         del df
#         gc.collect()
#     return fulldata

In [70]:
# %%time
# test = readAllData(n=5,name = 'test0')

loading data : test00


/home/baitong/tensorflow/lib/python3.6/site-packages/IPython/core/magic.py:187: DtypeWarning: Columns (43,45,51,53,62,63,71,76) have mixed types. Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


loading data : test01


/home/baitong/tensorflow/lib/python3.6/site-packages/IPython/core/magic.py:187: DtypeWarning: Columns (1,3,8,17,43,45,49,51,53,62,63,64,71,76) have mixed types. Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


loading data : test02


/home/baitong/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


loading data : test03
loading data : test04
CPU times: user 7.03 s, sys: 576 ms, total: 7.61 s
Wall time: 7.76 s


In [71]:
# train.columns

In [75]:
# f = ['visitId','totals_totalTransactionRevenue','hits_transaction.transactionId','hits_item.transactionId']

In [76]:
# test = test.drop(f,axis =1)

In [77]:
# test.to_csv(path+'test_combine.csv',index = False)

In [85]:
%%time
train = pd.read_csv(path+'train_combine.csv',dtype={'fullVisitorId': 'str'})

<string>:2: DtypeWarning: Columns (3,6,20,22,24,30,40,41,42,49,59,79) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 15.1 s, sys: 904 ms, total: 16 s
Wall time: 16 s


In [89]:
users_period = [
    (20160801, 20170115),
    (20170115, 20170630),
    (20170701, 20171215),
    (20171216, 20180601),
    # Using the same months!
    (20170501, 20171015),
]

predict_period = [
    (20170301, 20170430),
    (20170715, 20170915),
    (20180201, 20180331),
    (20180715, 20180915),
    # Using the same months!
    (20171201, 20180131),
]

In [93]:
train = train[train['date']!='No']
train['date'] = train['date'].astype("float")

In [95]:
revenues_list = []

for i in range(5):
    print("\nPeriod", i+1)
    a, b = users_period[i]
    batch = train[(train.date >= a) & (train.date <= b)]
    batch_visitors = batch.fullVisitorId.unique()
    print("There are", len(batch_visitors), "visitors in 5.5 months")
    
    c, d = predict_period[i]
    pred = train[(train.date >= c) & (train.date <= d)]
    pred_visitors = pred.fullVisitorId.unique()
    print("There are", len(pred_visitors), "visitors in 2 months")
    # Returning visitors
    same_visitors = np.intersect1d(batch_visitors, pred_visitors)
    print("{} visitors returned or {:.2f}%".format(len(same_visitors), 100*len(same_visitors)/len(batch_visitors)))
    # Returning visitors revenue
    with_rev = pred[(pred.fullVisitorId.isin(same_visitors)) & (pred.totals_transactionRevenue > 0)]
    print("And only {} returning visitors have revenue or {:.2f}% from total".format(len(with_rev), 100*len(with_rev)/len(batch_visitors)))
    print("The total revenue for this users is U$$ {:.2f}".format(with_rev.totals_transactionRevenue.sum()/1000000))
    revenues_list.append(with_rev.copy(deep=True))


Period 1
There are 377186 visitors in 5.5 months
There are 111299 visitors in 2 months
1871 visitors returned or 0.50%
And only 210 returning visitors have revenue or 0.06% from total
The total revenue for this users is U$$ 102504.27

Period 2
There are 286797 visitors in 5.5 months
There are 129144 visitors in 2 months
3285 visitors returned or 1.15%
And only 357 returning visitors have revenue or 0.12% from total
The total revenue for this users is U$$ 88637.14

Period 3
There are 384946 visitors in 5.5 months
There are 140808 visitors in 2 months
2943 visitors returned or 0.76%
And only 148 returning visitors have revenue or 0.04% from total
The total revenue for this users is U$$ 15886.58

Period 4
There are 297074 visitors in 5.5 months
There are 0 visitors in 2 months
0 visitors returned or 0.00%
And only 0 returning visitors have revenue or 0.00% from total
The total revenue for this users is U$$ 0.00

Period 5
There are 329636 visitors in 5.5 months
There are 144049 visitors i

In [110]:
def splitByDate(df,users_period,predict_period):
    print('split by date...')
    print('users_period is ',users_period[0],'======>',users_period[1])
    batch = train[(df.date >= users_period[0]) & (df.date <= users_period[1])]
    print('predict_period is ',predict_period[0],'======>',predict_period[1])
    pred = train[(df.date >= predict_period[0]) & (df.date <= predict_period[1])]
    print("-----------------done--------------------")
    return batch,pred

In [170]:
train['fullVisitorId'].astype("str")
batch1,pred1 = splitByDate(train,users_period[0],predict_period[0])
batch2,pred2 = splitByDate(train,users_period[1],predict_period[1])
batch3,pred3 = splitByDate(train,users_period[2],predict_period[2])
batch4,pred4 = splitByDate(train,users_period[4],predict_period[4])

split by date...
users_period is  20160801 ======> 20170115
predict_period is  20170301 ======> 20170430
-----------------done--------------------
split by date...
users_period is  20170115 ======> 20170630
predict_period is  20170715 ======> 20170915
-----------------done--------------------
split by date...
users_period is  20170701 ======> 20171215
predict_period is  20180201 ======> 20180331
-----------------done--------------------
split by date...
users_period is  20170501 ======> 20171015
predict_period is  20171201 ======> 20180131
-----------------done--------------------


True    5924
Name: isIdhavetest, dtype: int64
False    417184
True       7962
Name: isIdhavetest, dtype: int64


(5924, 425146)

In [174]:
# pred4.info()

Index(['0000049363351866189', '0000062267706107999', '0000085059828173212',
       '0000085840370633780', '0000091131414287111', '0000118334805178127',
       '0000166374699289385', '000018122977590134', '0000197671390269035',
       '000020731284570628',
       ...
       '9999773079368714197', '9999775084144044428', '9999799804624735268',
       '9999803509476553670', '9999824060591971928', '9999825631375939274',
       '9999860794386137754', '9999905960465191827', '9999944980984230465',
       '9999997304197521748'],
      dtype='object', name='fullVisitorId', length=329636)

329636

In [209]:
def extractTarget(df,user_period,pred_period,name):
#     print(user_period)
    batch,pred = splitByDate(df,user_period,pred_period)
    batch,pred = batch.copy(),pred.copy()
    print("提取交集...")
    ####提取交集 ： 即查看预测日期区间与训练集日期区间的用户交集，这个交集是二次光顾商店的人群
    pred["isIdhavetest"] = pred['fullVisitorId'].isin(batch['fullVisitorId'])
#     print(pred["isIdhavetest"].value_counts())
    pred = pred[pred["isIdhavetest"]==True]
    batch["isIdhavetest"]= batch['fullVisitorId'].isin(pred['fullVisitorId'])
#     print(batch["isIdhavetest"].value_counts())
     ####处理标签
    print("处理标签...")
    pred["isIdhavetest"] = pred["isIdhavetest"].apply(lambda x: 1 if x==True else 0)
    batch["isIdhavetest"] = batch["isIdhavetest"].apply(lambda x: 1 if x==True else 0)
    pred['target'] =pred['totals_transactionRevenue']
    pred = pred[['target','fullVisitorId',"isIdhavetest",'totals_transactionRevenue']]
    batch['target'] = 0
    batch = batch[['target','fullVisitorId',"isIdhavetest"]]
    ####按用户ID排序
    print("按用户ID排序并赋值...")
    pred = pred.groupby('fullVisitorId').sum()
    batch = batch.groupby('fullVisitorId').sum()
    tmp1 =pd.DataFrame()
    tmp2 = pd.DataFrame()
    tmp1['target'] = batch['target'][batch["isIdhavetest"]>=1] 
    tmp2['target'] = batch['target'][batch["isIdhavetest"]==0] 
    tmp1['target'] = pred['totals_transactionRevenue']#获取回头客的消费记录
    tmp2['target'] = 0#非回头客默认置零
    tmp = pd.concat([tmp1, tmp2])
    tmp.sort_index(inplace=True)
    print("保存...文件名: "+name+'.csv')
    batch['target'] = tmp['target']
    batch[['target']].to_csv('/home/baitong/pywork/RevenuePrediction/'+name+'.csv', index=True)
#     print(batch['target'].value_counts())
    print('------------------结束--------------------')

In [210]:
extractTarget(train,users_period[0],predict_period[0],'target1')
extractTarget(train,users_period[1],predict_period[1],'target2')
extractTarget(train,users_period[2],predict_period[2],'target3')
extractTarget(train,users_period[4],predict_period[4],'target4')

split by date...
users_period is  20160801 ======> 20170115
predict_period is  20170301 ======> 20170430
-----------------done--------------------
提取交集...
处理标签...
按用户ID排序并赋值...
保存...文件名: target1.csv
------------------结束--------------------
split by date...
users_period is  20170115 ======> 20170630
predict_period is  20170715 ======> 20170915
-----------------done--------------------
提取交集...
处理标签...
按用户ID排序并赋值...
保存...文件名: target2.csv
------------------结束--------------------
split by date...
users_period is  20170701 ======> 20171215
predict_period is  20180201 ======> 20180331
-----------------done--------------------
提取交集...
处理标签...
按用户ID排序并赋值...
保存...文件名: target3.csv
------------------结束--------------------
split by date...
users_period is  20170501 ======> 20171015
predict_period is  20171201 ======> 20180131
-----------------done--------------------
提取交集...
处理标签...
按用户ID排序并赋值...
保存...文件名: target4.csv
------------------结束--------------------


In [67]:
# train['date'] = train['date'].astype("float")

# dev_df = train[(train['date']<=20171015) & (train['date']>=(20170501))]
# val_df = train[(train['date']>=20171201) & (train['date']<=(20180131))]
# len(dev_df),len(val_df)